# 전통적 시계열 모델링

   ## 유통매장 수요량 예측

* 비즈니스 현황
    * 고객사는 A 유통회사의 a 매장 입니다.
    * a 매장에서 주력상품인 a01에 대한 재고 최적화를 위해 수요량을 예측하고자 합니다.
    * 최근 경쟁사의 매장이 가까운 거리에 오픈하였고, 유사한 상품에 대한 공격적인 마케팅을 펼치고 있습니다.
* 발주 최적화를 위한 수요량 예측
    * 일마감 이후, 발주량을 결정할 때, 예측된 수요량이 필요합니다.
    * 발주후 입고까지는 2일의 기간이 걸립니다.
    * 예를 들면
        * 2019년 6월 1일 저녁 10시 일마감 직후, 6월 3일의 수요량을 예측해야 합니다.
        * 예측된 수요량과 현 재고, 안전재고량 등을 감안하여 발주량은 결정되고,
        * 발주 수량은 6월3일 새벽에 매장에 입고되어 6월3일에 판매할 수 있습니다.
* 발주 최적화를 위한 수요량 예측 : 2일후의 판매량을 예측

![](https://www.artefact.com//wp-content/uploads/2021/08/GettyImages-1295864156-scaled.jpg)

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) 함수 생성

* 결과 시각화

In [ ]:
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

* 잔차분석

In [ ]:
def residual_diag(residuals, lags = 30) :
    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

## (3) Data Loading

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/retail_demand2.csv'
data = pd.read_csv(path)
data = data.loc[data['date'].between('2013-06-01', '2015-03-02')].reset_index(drop = True)
data

* 변수 설명

    * date : 날짜
    * item : 상품코드 (여기서는 한가지 상품만 있음)
    * sales : A유통회사 a 매장 판매량 ==> target
    * tot_sales : A유통회사 전체 판매량
    * comp_sales : 인근에 위치한 B유통회사 b 매장 판매량

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

# 2.기본 전처리

## (1) y 만들기

* 2일 후 수요량을 예측하기 위한 y 만들기

In [ ]:
data['y'] = data['sales'].shift(-2)
display(data.head())
display(data.tail())

In [ ]:
# 마지막 두 행은 삭제
data.dropna(axis = 0, inplace = True)
data.tail()

## (2) 데이터 분할

### 1) x, y 나누기

In [ ]:
target = 'y'

x = data.drop([target, 'date'], axis = 1) #제거할 때, date도 제거
y = data.loc[:, target]

### 2) train, val 분할
* 최근 30일 데이터를 Validation Set으로 지정

In [ ]:
val_size = 30
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = val_size, shuffle = False)

# 3.모델링1 : ARIMA

## (1) y 값 살펴보기

In [ ]:
residual_diag( )

## (2) 모델링 : 초기모델

* p, d, q 값을 어떻게 정해야 할까요?
* 초기 차수를 1,1,1로 정하고 모델링을 수행해 봅시다.

### 1) 학습

* sm.tsa.SARIMAX(train, order=(p,d,q)).fit()
    * 모델 선언시 train이 포함
    * .fit()으로 학습.

In [ ]:
# ARIMA 모델링
m1 = sm.tsa.SARIMAX(      ).fit()

### 2) 평가

#### ① 잔차진단

* 모델.resid : 잔차를 뽑을 수 있습니다.
* 위에서 만든 함수 residual_diag 를 사용하여 잔차진단을 해 봅시다.

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수
* SARIMAX 모델.aic로 쉽게 통계량을 구할 수 있습니다.

#### ③ Validation

시계열 데이터로 실제값과 예측값에 대해 비교하여 그래프를 그려봅시다.

* 결과 시각화

## (3) 하이퍼파라미터 튜닝

실제로 p, d, q를 찾는 과정은 마치 Grid Search 처럼 값을 조금씩 조정해가며  최적의 모델을 찾아가는 과정과 유사합니다.


### 1) 학습

In [ ]:
from itertools import product

* 값의 범위 지정
    * 시간이 많이 소요될 수 있으니 범위를 적절하게 지정하세요.

In [ ]:
# product 함수를 이용하여 값의 조합을 구성
p =
q =
d =
iter = list(product(p,d,q))
iter

* 튜닝

In [ ]:
mae, aic = [],[]



* 튜닝 결과 조회

In [ ]:
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})
result

In [ ]:
display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
# 최적의 하이퍼파라미터로 모델 생성



### 2) 평가

#### ① 잔차진단

* residual_diag

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

#### ③ Validation

* 결과 시각화

# 4.모델링2 : SARIMA

## (1) 모델링 : 초기모델

### 1) 학습
* Seasonal 하이퍼파라미터(P, D, Q, m) 초기 값으로 1, 1, 1, 7로 모델링

In [ ]:
# SARIMA 모델링
m2 = sm.tsa.SARIMAX(y_train, order=(4,1,3), seasonal_order=(1,1,1,7)).fit()

### 2) 평가

#### ① 잔차진단

In [ ]:
residuals = m2.resid
residual_diag(residuals, lags = 30)

#### ② AIC

In [ ]:
print('model2_0 AIC :', m2.aic)

#### ③ Validation

In [ ]:
pred = m2.forecast(val_size)

print('MAE  : ', mean_absolute_error(y_val, pred))
print('MAPE : ', mean_absolute_percentage_error(y_val, pred))
print('R2   : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

## (2) 하이퍼파라미터 튜닝

### 1) 학습
* 다음의 범위로 하이퍼 파라미터 튜닝을 진행합시다.(범위를 더 늘리면 시간이 많이 걸립니다.)

    * P : 1,2,3
    * Q : 1,2,3
    * D : 1
    * m : 7

In [ ]:
result = pd.DataFrame({'params(P,D,Q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
# 최적의 하이퍼파라미터로 모델 생성



### 2) 평가

#### ① 잔차진단

* residual_diag

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

#### ③ Validation

* 결과 시각화

# 5.모델링3 : SARIMAX
* 여기서는 feature에 대한 추가 없이, 기존 feature로만 모델링을 수행합니다.

## (1) 모델링

### 1) 학습

In [ ]:
# 4번에서 찾은 최적의 하이퍼파라미터 + x_train 이용하여 모델 생성하기



### 2) 평가

#### ① 잔차진단

* residual_diag

#### ② AIC

#### ③ Validation
SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함.

* 결과 시각화

# 6.예측결과 저장
* joblib을 이용하여
* 가장 성능이 좋았던 모델의 예측 결과를 저장합니다.


In [ ]:
import joblib

joblib.dump(    , 'pred_22.pkl')

**저장된 파일을 다운로드 받으세요.**
* 왼쪽 파일 탭 > 저장한 파일 오른쪽 클릭 > 다운로드